In [1]:
import os
import pickle
import random
import sys
import warnings

# sys.path.append(os.path.join("../models"))
# import conv_lstm
# from conv_lstm import ConvLSTM

import torch
# from src.model_utils import custom_multiclass_report, CroplandDataModule_LSTM, Crop_LSTM, Crop_PL
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Dataset

import pytorch_lightning as pl
import torchmetrics
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor

/home/crop_dev/miniconda/envs/crop_env/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/crop_dev/miniconda/envs/crop_env/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Read dictionary pkl file
with open(os.path.join('..', 'data', 'processed_files', 'pkls', 'X_FR_ROS_lstm.pkl'), "rb") as fp:
    X = pickle.load(fp)

with open(os.path.join('..', 'data', 'processed_files', 'pkls', 'y_FR_ROS_lstm.pkl'), "rb") as fp:
    y = pickle.load(fp)

In [3]:
class CroplandDataset(Dataset):
    def __init__(self, X_monthly, X_static, y):
        self.X_monthly = X_monthly  
        self.X_static = X_static
        self.y = y 

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x_monthly = self.X_monthly[idx]
        x_static = self.X_static[idx]
        target = self.y[idx]
        
        return (x_monthly, x_static), target

In [4]:
batch_size =512
# X_train = torch.Tensor(X['Train'])
# y_train = torch.Tensor(y['Train'])
# X_test = torch.Tensor(X['Test'])
# y_test = torch.Tensor(y['Test'])

# train_dataset = TensorDataset((X_train_m, X_train_s),y_train)
# test_dataset = TensorDataset(X_test,y_test)

trainloader = torch.utils.data.DataLoader(CroplandDataset(torch.FloatTensor(X["Train"][0]),
                                                        torch.FloatTensor(X["Train"][1]),
                                                        torch.FloatTensor(y["Train"])),
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)

testloader = torch.utils.data.DataLoader(CroplandDataset(torch.FloatTensor(X["Test"][0]),
                                                        torch.FloatTensor(X["Test"][1]),
                                                        torch.FloatTensor(y["Test"])),
                                        batch_size=batch_size,
                                        shuffle=False,
                                        num_workers=2)

# testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
#                                         shuffle=False, num_workers=2)
# X_train.size()

In [5]:
class ConvLSTMCell(nn.Module):
    """
    Initialize ConvLSTM cell.

    Parameters
    ----------
    input_dim (int): Number of channels of input tensor.
    hidden_dim (int): Number of channels of hidden state.
    kernel_size (int): Size of the convolutional kernel.
    bias (bool): Whether to add the bias.
    """
    
    def __init__(self,
        input_dim,
        hidden_dim,
        kernel_size,
        bias):

        super(ConvLSTMCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2
        self.bias = bias

        self.conv = nn.Conv1d(in_channels=self.input_dim + self.hidden_dim,
                            out_channels=4 * self.hidden_dim,
                            kernel_size=self.kernel_size,
                            padding=self.padding,
                            bias=self.bias)

    def forward(self, input, cur_state):
        h_cur, c_cur = cur_state
        combined = torch.cat([input, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, length):
        return (torch.zeros(batch_size, self.hidden_dim, length, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, length, device=self.conv.weight.device))

In [6]:
class CropConvLSTM(nn.Module):

    """
    A PyTorch module implementing a Crop Conv LSTM network.
    
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        n_layers: Number of LSTM layers stacked on each other
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers

    Input:
        A tensor of size B, T, C
    Output:
        A tuple of two lists of length n_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    """

    def __init__(self,
        input_dim: int,
        hidden_dim: int,
        kernel_size: tuple,
        n_layers: int,
        n_classes: int,
        input_len_monthly: int,
        seq_len: int,
        input_len_static: int,
        bias: bool=True,
        return_all_layers: bool=False
        ) -> None:
        super(CropConvLSTM, self).__init__()

        
        
        assert (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))),'`kernel_size` must be tuple or list of tuples' 
        # self._check_kernel_size_consistency(kernel_size)
        
        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == n_layers
        kernel_size = self._extend_for_multilayer(kernel_size, n_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, n_layers)
        assert len(kernel_size) == len(hidden_dim) == n_layers, 'Inconsistent list length.'

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.n_layers = n_layers
        self.bias = bias
        self.return_all_layers = return_all_layers
        self.n_classes = n_classes
        self.input_len_monthly = input_len_monthly
        self.seq_len = seq_len
        self.input_len_static = input_len_static
        
        cell_list = []
        for i in range(0, self.n_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                        hidden_dim=self.hidden_dim[i],
                                        kernel_size=self.kernel_size[i],
                                        bias=self.bias))
        self.cell_list = nn.ModuleList(cell_list)
        
        self.flatten = nn.Flatten()
        self.net = nn.Sequential(
            nn.Linear(self.hidden_dim[0]*self.seq_len*self.input_len_monthly+self.input_len_static, self.hidden_dim[0]*self.seq_len),
            nn.BatchNorm1d(self.hidden_dim[0]*self.seq_len),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(self.hidden_dim[0]*self.seq_len, self.seq_len),
            nn.BatchNorm1d(self.seq_len),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(self.seq_len, self.n_classes)
        )
        
    def forward(self, input, hidden_state=None):
        """
        Args:
        input (tuple):  input[0] is a tensor of shape (batch_size, sequence_length, input_size) containing the monthly input sequence.
                        input[1] is a tensor of shape (batch_size, input_size) containing the static input sequence.

        Returns
        -------
        last_state_list, layer_output
        """
        input_monthly = input[0][:, None, :, :]  #fictional dimension added. will be used as channnels
        input_static = input[1]
        b = input_monthly.size()[0]

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            hidden_state = self._init_hidden(batch_size=b, length=self.input_len_monthly)

        layer_output_list = []
        last_state_list = []
        cur_layer_input = input_monthly

        for layer_idx in range(self.n_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(self.seq_len):
                h, c = self.cell_list[layer_idx](input = cur_layer_input[:, :, t, :],
                                                cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        output_monthly = self.flatten(layer_output_list[0])        
        output = self.net(torch.cat((output_monthly, input_static), dim=1))

        return output

    def _init_hidden(self, batch_size, length):
        init_states = []
        for i in range(self.n_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, length))
        return init_states

    # @staticmethod
    # def _check_kernel_size_consistency(kernel_size):
    #     if not (isinstance(kernel_size, tuple) or
    #             (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
    #         raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, n_layers):
        if not isinstance(param, list):
            param = [param] * n_layers
        return param


In [8]:
network = CropConvLSTM(
    input_dim=1, #fictional dimension. will be used as channnels
    hidden_dim=16,
    kernel_size=(3,),
    n_layers=1,
    n_classes = 4,
    input_len_monthly = X['Train'][0].shape[2],
    seq_len = X['Train'][0].shape[1],
    input_len_static = X['Train'][1].shape[1],
    bias=False,
    return_all_layers=False
    )

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        output = network(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

# preds = nn.Softmax(output)
# f1 = torchmetrics.F1Score(task="multiclass", num_classes=4, average="macro")
# f1(preds, labels)
print('Finished Training')

Finished Training


In [10]:

# %%
# Save the module to a file
model_filename = os.path.join("..", "results", "pickle_models", "lstm_pytorch_ROS.pkl")
torch.save(network, model_filename)

In [51]:
# prepare to count predictions for each class
classes = (0,1,2,3)
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
f1 = torchmetrics.F1Score(task = 'multiclass', num_classes = 4, average = 'macro')

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = network(images)
        _, predictions = torch.max(outputs, 1)
        labels = torch.argmax(labels, 1)
        f1.update(predictions, labels)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            class_label = torch.argmax(label)
            if class_label == int(prediction):
                correct_pred[classes[class_label]] += 1
            total_pred[classes[class_label]] += 1
total_f1 = f1.compute()
print('f1 equals', total_f1)
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname} is {accuracy:.1f} %')

f1 equals tensor(0.2898)
Accuracy for class: 0 is 30.6 %


ZeroDivisionError: float division by zero